In [12]:
%pip install pdf2image

  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from pdf2image import convert_from_path

def pdf_to_images(pdf_path, output_dir, dpi=300):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Convert PDF to a list of images
    images = convert_from_path(pdf_path, dpi)
    
    # Save images to the output directory
    for i, image in enumerate(images):
        image_path = os.path.join(output_dir, f"{os.path.basename(pdf_path)}_{i+1}.jpg")
        image.save(image_path, "JPEG")
    
    return images

# Specify the directory containing PDF files
pdf_directory = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13\eu-dataset'

# Specify the directory to save the converted images
output_directory = os.path.join(pdf_directory, "eu-dataset-images")

# Process each PDF file in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        pdf_to_images(pdf_path, output_directory)

print("PDF to images conversion complete.")


PDF to images conversion complete.


In [7]:
%pip install ultralyticsplus
%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
from ultralyticsplus import YOLO, render_result
from PIL import Image
import numpy as np

# Load YOLOv8m model
model = YOLO('keremberke/yolov8m-table-extraction')

# Set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.70  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # Maximum number of detections per image

# Define path to the directory containing images
input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13\eu-dataset\eu-dataset-images'

# Create directory if not exists to store cropped table images
cropped_output_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images'
os.makedirs(cropped_output_dir, exist_ok=True)

# Loop through all images in the directory
for filename in os.listdir(input_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        img_path = os.path.join(input_dir, filename)
        img = np.array(Image.open(img_path))

        # Perform inference
        results = model.predict(img)

        # Check if any tables are detected
        if len(results) > 0 and len(results[0].boxes.data.numpy()) > 0:
            # Get the bounding box coordinates of the first detected table
            x1, y1, x2, y2, _, _ = tuple(int(item) for item in results[0].boxes.data.numpy()[0])

            # Cropping the detected table region from the original image
            cropped_image = img[y1:y2, x1:x2]
            cropped_image = Image.fromarray(cropped_image)

            # Save the cropped image
            cropped_output_path = os.path.join(cropped_output_dir, f'cropped_{filename}')
            cropped_image.save(cropped_output_path)

            # Display the path of the saved cropped image
            print(f"Cropped image saved to: {cropped_output_path}")
        else:
            # Display a message if no tables are detected in the image or if the first item has no detections
            print(f"No tables detected in image: {filename}")



0: 640x480 1 borderless, 861.1ms
Speed: 0.0ms preprocess, 861.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
Cropped image saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images\cropped_eu-001.pdf_1.jpg

0: 640x480 3 borderlesss, 691.0ms
Speed: 0.0ms preprocess, 691.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
Cropped image saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images\cropped_eu-002.pdf_1.jpg

0: 640x480 (no detections), 732.3ms
Speed: 0.0ms preprocess, 732.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
No tables detected in image: eu-002.pdf_2.jpg

0: 640x480 2 borderlesss, 714.7ms
Speed: 0.0ms preprocess, 714.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
Cropped image saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images\cropped_eu-003.pdf_1.jpg

0: 640x480 2 borderlesss, 697.6ms
Speed: 0.0ms preprocess, 697.6ms

In [13]:
import os
from ultralyticsplus import YOLO, render_result
from PIL import Image
import numpy as np

# Load YOLOv8m model
model = YOLO('keremberke/yolov8m-table-extraction')

# Set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.70  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # Maximum number of detections per image

# Define path to the directory containing images
input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13\eu-dataset\eu-dataset-images'

# Create directories if they don't exist to store cropped table images
bordered_output_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images_bordered'
borderless_output_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images_borderless'
os.makedirs(bordered_output_dir, exist_ok=True)
os.makedirs(borderless_output_dir, exist_ok=True)

# Loop through all images in the directory
for filename in os.listdir(input_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        img_path = os.path.join(input_dir, filename)
        img = np.array(Image.open(img_path))

        # Perform inference
        results = model.predict(img)

        # Check if any tables are detected
        if len(results) > 0 and len(results[0].boxes) > 0:
            for box in results[0].boxes:
                # Get the bounding box coordinates and the label
                x1, y1, x2, y2, conf, label = box

                # Convert coordinates to integers
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

                # Cropping the detected table region from the original image
                cropped_image = img[y1:y2, x1:x2]
                cropped_image = Image.fromarray(cropped_image)

                # Save the cropped image based on the label
                if label == 0:  # Assuming label 0 is for borderless tables
                    cropped_output_path = os.path.join(borderless_output_dir, f'cropped_{filename}')
                else:  # Assuming any other label is for bordered tables
                    cropped_output_path = os.path.join(bordered_output_dir, f'cropped_{filename}')

                cropped_image.save(cropped_output_path)

                # Display the path of the saved cropped image
                print(f"Cropped image saved to: {cropped_output_path}")
        else:
            # Display a message if no tables are detected in the image or if the first item has no detections
            print(f"No tables detected in image: {filename}")



0: 640x480 1 borderless, 1065.5ms
Speed: 33.6ms preprocess, 1065.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 480)


ValueError: not enough values to unpack (expected 6, got 1)

In [29]:
import os
import cv2

def apply_dilation(img, kernel_size=5, iterations=1):
    return cv2.dilate(img, cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size)), iterations=iterations)

def create_binary_inverted_image(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply binary thresholding
    _, binary_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    return binary_img

def process_image_with_erosion_dilation(input_dir, output_dir, kernel_size=5, iterations=1):
    os.makedirs(output_dir, exist_ok=True)

    # Loop through all images in the directory
    for filename in os.listdir(input_dir):
        if (filename.endswith('.jpg') or filename.endswith('.png')):
            # Read the image
            img_path = os.path.join(input_dir, filename)
            img = cv2.imread(img_path)

            # Create binary inverted image
            binary_img = create_binary_inverted_image(img)

            # Apply dilation
            dilated_img = apply_dilation(binary_img, kernel_size, iterations)
            
            # Save the processed images with dilation
            output_path = os.path.join(output_dir, filename.replace('.jpg', '_dilated.jpg').replace('.png', '_dilated.png'))
            cv2.imwrite(output_path, dilated_img)

            # Display the path of the saved image
            print(f"Processed image with dilation saved to: {output_path}")

# Define the directory containing the images
input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images'

# Define the directory to save the processed images
output_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation'

# Define the parameters for dilation
kernel_size = 5
iterations = 1

# Process images with dilation
process_image_with_erosion_dilation(input_dir, output_dir, kernel_size, iterations)


Processed image with dilation saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation\cropped_eu-001.pdf_1_dilated.jpg
Processed image with dilation saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation\cropped_eu-002.pdf_1_dilated.jpg
Processed image with dilation saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation\cropped_eu-003.pdf_1_dilated.jpg
Processed image with dilation saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation\cropped_eu-003.pdf_2_dilated.jpg
Processed image with dilation saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation\cropped_eu-004.pdf_3_dilated.jpg
Processed image with dilation saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation\cropped_eu-005.pdf_3_dilated.jpg
Processed image with dilation saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_

In [30]:
import os
import cv2
import numpy as np

def preprocess_image(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to smooth the image (reduce noise)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Apply adaptive thresholding to binarize the image
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def detect_cells(img):
    # Find contours in the binary image
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_contours(img, contours):
    # Create a blank image to draw contours
    contour_img = np.zeros_like(img)
    # Draw contours on the blank image
    cv2.drawContours(contour_img, contours, -1, (255, 255, 255), 2)
    return contour_img

# Define the directory containing the images
input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation'

# Define the directory to save the processed images with contours
output_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_contours'
os.makedirs(output_dir, exist_ok=True)

# Loop through all images in the directory
for filename in os.listdir(input_dir):
    if (filename.endswith('.jpg') or filename.endswith('.png')):
        # Read the image
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path)
        
        # Preprocess the image
        preprocessed_img = preprocess_image(img)
        
        # Detect cells using contours
        contours = detect_cells(preprocessed_img)
        
        # Draw contours on the original image
        contour_img = draw_contours(img, contours)
        
        # Save the image with contours
        output_path = os.path.join(output_dir, filename.replace('.jpg', '_contours.jpg').replace('.png', '_contours.png'))
        cv2.imwrite(output_path, contour_img)
        
        # Display the path of the saved image
        print(f"Image with contours saved to: {output_path}")


Image with contours saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_contours\cropped_eu-001.pdf_1_dilated_contours.jpg
Image with contours saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_contours\cropped_eu-002.pdf_1_dilated_contours.jpg
Image with contours saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_contours\cropped_eu-003.pdf_1_dilated_contours.jpg
Image with contours saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_contours\cropped_eu-003.pdf_2_dilated_contours.jpg
Image with contours saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_contours\cropped_eu-004.pdf_3_dilated_contours.jpg
Image with contours saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_contours\cropped_eu-005.pdf_3_dilated_contours.jpg
Image with contours saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images

In [31]:
import os
import cv2
import numpy as np

def preprocess_image(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to smooth the image (reduce noise)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Apply adaptive thresholding to binarize the image
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def detect_cells(img):
    # Find contours in the binary image
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_bounding_boxes(original_img, contours):
    # Draw bounding boxes on the original image
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(original_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return original_img

# Define the directory containing the images
input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation'

# Define the directory to save the processed images with bounding boxes
output_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes'
os.makedirs(output_dir, exist_ok=True)

# Loop through all images in the directory
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Read the image
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path)
        
        # Preprocess the image
        preprocessed_img = preprocess_image(img)
        
        # Detect cells using contours
        contours = detect_cells(preprocessed_img)
        
        # Draw bounding boxes on the original image
        img_with_bboxes = draw_bounding_boxes(img.copy(), contours)
        
        # Save the image with bounding boxes
        output_path = os.path.join(output_dir, filename.replace('.jpg', '_with_bboxes.jpg').replace('.png', '_with_bboxes.png'))
        cv2.imwrite(output_path, img_with_bboxes)
        
        # Display the path of the saved image
        print(f"Image with bounding boxes saved to: {output_path}")


Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-001.pdf_1_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-002.pdf_1_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-003.pdf_1_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-003.pdf_2_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-004.pdf_3_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-005.pdf_3_dilated_with_bboxes.jpg
Imag

In [35]:
import os
import cv2
import numpy as np

def preprocess_image(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to smooth the image (reduce noise)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Apply adaptive thresholding to binarize the image
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def detect_cells(img):
    # Find contours in the binary image
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_bounding_boxes(original_img, contours):
    # Draw bounding boxes on the original image
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(original_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return original_img

# Define the directory containing the original images
original_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images'

# Define the directory containing the processed images (for contour detection)
input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_dilation'

# Define the directory to save the processed images with bounding boxes
output_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes'
os.makedirs(output_dir, exist_ok=True)

# Loop through all images in the processed images directory
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Read the processed image
        processed_img_path = os.path.join(input_dir, filename)
        processed_img = cv2.imread(processed_img_path)
        
        # Read the original image
        original_img_path = os.path.join(original_dir, filename.replace('_dilated', ''))
        original_img = cv2.imread(original_img_path)
        
        if original_img is None:
            print(f"Original image not found for {filename}")
            continue

        # Preprocess the processed image
        preprocessed_img = preprocess_image(processed_img)
        
        # Detect cells using contours
        contours = detect_cells(preprocessed_img)
        
        # Draw bounding boxes on the original image
        img_with_bboxes = draw_bounding_boxes(original_img.copy(), contours)
        
        # Save the image with bounding boxes
        output_path = os.path.join(output_dir, filename.replace('.jpg', '_with_bboxes.jpg').replace('.png', '_with_bboxes.png'))
        cv2.imwrite(output_path, img_with_bboxes)
        
        # Display the path of the saved image
        print(f"Image with bounding boxes saved to: {output_path}")


Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-001.pdf_1_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-002.pdf_1_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-003.pdf_1_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-003.pdf_2_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-004.pdf_3_dilated_with_bboxes.jpg
Image with bounding boxes saved to: C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_processed_images_with_bounding_boxes\cropped_eu-005.pdf_3_dilated_with_bboxes.jpg
Imag

TSR for Unbordered Tables

Smearing

In [7]:
import cv2
import numpy as np
import os

def preprocess_image(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply thresholding to get a binary image
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    return binary_image

def rlsa(binary_image, horizontal_threshold, vertical_threshold):
    def rlsa_horizontal(img, threshold):
        for row in range(img.shape[0]):
            run_start = -1
            for col in range(img.shape[1]):
                if img[row, col] == 255:
                    if run_start != -1:
                        run_length = col - run_start
                        if run_length <= threshold:
                            img[row, run_start:col] = 255
                    run_start = col
                elif run_start != -1 and col == img.shape[1] - 1:
                    run_length = col - run_start
                    if run_length <= threshold:
                        img[row, run_start:] = 255
        return img

    def rlsa_vertical(img, threshold):
        for col in range(img.shape[1]):
            run_start = -1
            for row in range(img.shape[0]):
                if img[row, col] == 255:
                    if run_start != -1:
                        run_length = row - run_start
                        if run_length <= threshold:
                            img[run_start:row, col] = 255
                    run_start = row
                elif run_start != -1 and row == img.shape[0] - 1:
                    run_length = row - run_start
                    if run_length <= threshold:
                        img[run_start:, col] = 255
        return img

    img_h = rlsa_horizontal(binary_image.copy(), horizontal_threshold)
    img_v = rlsa_vertical(binary_image.copy(), vertical_threshold)
    return np.minimum(img_h, img_v)

def process_directory(input_dir, output_dir, horizontal_threshold=10, vertical_threshold=10):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate over all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_dir, filename)
            
            # Preprocess the image to get a binary image
            binary_image = preprocess_image(image_path)
            
            # Apply RLSA to the binary image
            smeared_image = rlsa(binary_image, horizontal_threshold, vertical_threshold)
            
            # Save the smeared image to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, smeared_image)
            print(f"Processed and saved: {output_path}")

def main():
    # Input and output directories
    input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images'
    output_dir = r'C:\Users\User\Desktop\Data Analysis\output_smeared_images'
    
    # Process all images in the input directory and save the results in the output directory
    process_directory(input_dir, output_dir)

# Run the main function
if __name__ == "__main__":
    main()


Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-001.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-002.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-003.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-003.pdf_2.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-004.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-005.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-005.pdf_5.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-005.pdf_9.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-006.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_

In [8]:
import cv2
import numpy as np
import os

def preprocess_image(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply thresholding to get a binary image
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    return binary_image

def smear_foreground(binary_image, kernel_size=(5, 5)):
    # Create a rectangular kernel for dilation
    kernel = np.ones(kernel_size, np.uint8)
    
    # Dilate the binary image
    smeared_image = cv2.dilate(binary_image, kernel, iterations=1)
    
    return smeared_image

def process_directory(input_dir, output_dir, kernel_size=(5, 5)):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate over all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_dir, filename)
            
            # Preprocess the image to get a binary image
            binary_image = preprocess_image(image_path)
            
            # Smear the foreground of the binary image
            smeared_image = smear_foreground(binary_image, kernel_size)
            
            # Save the smeared image to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, smeared_image)
            print(f"Processed and saved: {output_path}")

def main():
    # Input and output directories
    input_dir = r'C:\Users\User\Desktop\Data Analysis\ICDAR-13_eu_cropped_table_images'
    output_dir = r'C:\Users\User\Desktop\Data Analysis\output_smeared_images'
    
    # Process all images in the input directory and save the results in the output directory
    process_directory(input_dir, output_dir)

# Run the main function
if __name__ == "__main__":
    main()


Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-001.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-002.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-003.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-003.pdf_2.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-004.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-005.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-005.pdf_5.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-005.pdf_9.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_smeared_images\cropped_eu-006.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_

Bounding Box Detection

In [6]:
import cv2
import numpy as np
import os

def detect_bounding_boxes(image_path):
    # Load the smeared image in grayscale
    smeared_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Find contours in the smeared image
    contours, _ = cv2.findContours(smeared_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create an empty image for drawing bounding boxes
    bounding_box_image = cv2.cvtColor(smeared_image, cv2.COLOR_GRAY2BGR)
    
    # Draw bounding boxes around each contour
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(bounding_box_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return bounding_box_image

def process_directory(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate over all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_dir, filename)
            
            # Detect bounding boxes from the smeared image
            bounding_box_image = detect_bounding_boxes(image_path)
            
            # Save the bounding box image to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, bounding_box_image)
            print(f"Processed and saved: {output_path}")

def main():
    # Input and output directories
    input_dir = r'C:\Users\User\Desktop\Data Analysis\output_smeared_images'
    output_dir = r'C:\Users\User\Desktop\Data Analysis\output_bounding_boxes'
    
    # Process all images in the input directory and save the results in the output directory
    process_directory(input_dir, output_dir)

# Run the main function
if __name__ == "__main__":
    main()


Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-001.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-002.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-003.pdf_1.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-003.pdf_2.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-004.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-005.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-005.pdf_5.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-005.pdf_9.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_bounding_boxes\cropped_eu-006.pdf_3.jpg
Processed and saved: C:\Users\User\Desktop\Data Analysis\output_